In [ ]:
import firedrake
mesh = firedrake.UnitSquareMesh(
    32, 32, diagonal='crossed'
)

In [ ]:
from firedrake import inner, exp, Constant
x = firedrake.SpatialCoordinate(mesh)

ξ_1 = Constant((0.25, 0.25))
ξ_2 = Constant((0.75, 0.75))

r_1 = Constant(1 / 8)
r_2 = Constant(1 / 8)

ϕ_1 = exp(-inner(x - ξ_1, x - ξ_1) / r_1**2)
ϕ_2 = exp(-inner(x - ξ_2, x - ξ_2) / r_2**2)

I = firedrake.Identity(2)
ω = Constant(0.5)
J = firedrake.as_tensor([[0, +ω], [-ω, 0]])

s = Constant(50.0)
U = s * (
    ϕ_1 * (I + J) * (x - ξ_1) -
    ϕ_2 * (I + J) * (x - ξ_2)
)

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, 'CG', 2)
u = firedrake.interpolate(U, V)

In [ ]:
from firedrake import grad, div
Σ = firedrake.TensorFunctionSpace(mesh, 'CG', 2)
F = firedrake.interpolate(grad(U), Σ)

Q = firedrake.FunctionSpace(mesh, 'CG', 2)
div_u = firedrake.interpolate(div(U), Q)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
streamlines = firedrake.streamplot(u, resolution=1/40, axes=axes)
fig.colorbar(streamlines);

In [ ]:
def f(t, x):
    try:
        return u(x)
    except firedrake.PointNotInDomainError:
        return np.array([0.0, 0.0])
    
def jac(t, x):
    try:
        return F(x)
    except  firedrake.PointNotInDomainError:
        return np.array([[0.0, 0.0], [0.0, 0.0]])

In [ ]:
import numpy as np
import scipy.integrate

def integrate_grid(xs, ys, ts):
    results = np.zeros((len(xs), len(ys), 2, len(ts)))
    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            result = scipy.integrate.solve_ivp(
                f,
                jac=jac,
                t_span=(ts[0], ts[-1]),
                dense_output=True,
                y0=(x, y),
                method='LSODA',
                atol=1e-6,
            )
            results[i, j, :, :] = result.sol(ts)
            
    return results

In [ ]:
xs = np.linspace(0.2, 0.3, 6)
ys = np.linspace(0.2, 0.3, 6)
ts = np.linspace(0, 0.1, 101)
results1 = integrate_grid(xs, ys, ts)

In [ ]:
xs = np.linspace(0.675, 0.825, 6)
ys = np.linspace(0.675, 0.825, 6)
results2 = integrate_grid(xs, ys, ts)

In [ ]:
%%capture
fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.set_xlim((0, 1))
axes.set_ylim((0, 1))
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
firedrake.tripcolor(div_u, cmap='RdBu_r', axes=axes)

lines1 = [
    axes.plot(*results1[:, 0, :, 0].T, color='k')[0],
    axes.plot(*results1[:, -1, :, 0].T, color='k')[0],
    axes.plot(*results1[0, :, :, 0].T, color='k')[0],
    axes.plot(*results1[-1, :, :, 0].T, color='k')[0],
]

lines2 = [
    axes.plot(*results2[:, 0, :, 0].T, color='k')[0],
    axes.plot(*results2[:, -1, :, 0].T, color='k')[0],
    axes.plot(*results2[0, :, :, 0].T, color='k')[0],
    axes.plot(*results2[-1, :, :, 0].T, color='k')[0],
]

In [ ]:
from matplotlib.animation import FuncAnimation

def animate(index):
    global lines1, results2
    lines1[0].set_data(*results1[:, 0, :, index].T)
    lines1[1].set_data(*results1[:, -1, :, index].T)
    lines1[2].set_data(*results1[0, :, :, index].T)
    lines1[3].set_data(*results1[-1, :, :, index].T)

    lines2[0].set_data(*results2[:, 0, :, index].T)
    lines2[1].set_data(*results2[:, -1, :, index].T)
    lines2[2].set_data(*results2[0, :, :, index].T)
    lines2[3].set_data(*results2[-1, :, :, index].T)

animation = FuncAnimation(fig, animate, frames=list(range(len(ts))), interval=1e3/30)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())